YU GAN OH - GPT

In [ ]:
import torch
torch.cuda.is_available()


In [5]:
!conda install -y pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
!conda install -y scikit-learn
!conda install -y matplotlib pandas

Channels:
 - pytorch
 - nvidia
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Channels:
 - defaults
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Channels:
 - defaults
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/gxo

  added / updated specs:
    - matplotlib
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bottleneck-1.3.7           |  py312ha883a20_0         140 KB
    fontconfig-2.14.1          |       hef1e5e3_0         284 KB
    icu-58.2                   |       he6710b0_3        10.5 MB
    libxkbcommon-1.0.1         |       hfa300c1_0         483 KB
    libxml2-2.9.14             |       h74e7548_0         718 KB
    num

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.stats import entropy
from scipy.linalg import sqrtm
import os
import matplotlib.pyplot as plt

In [8]:


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 50
batch_size = 64
sample_dir = 'samples'

# Create directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Generator network
class Generator(nn.Module):
    def __init__(self, latent_size, hidden_size, image_size):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, image_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)


# Discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)


In [9]:
# Initialize networks
generator = Generator(latent_size, hidden_size, image_size).to(device)
discriminator = Discriminator(image_size, hidden_size).to(device)

# Loss function
criterion = nn.BCELoss()

# Optimizers
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002)


In [13]:
# Load data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
mnist_data = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
data_loader = DataLoader(dataset=mnist_data, batch_size=batch_size, shuffle=True)


In [14]:

# Define the sample noise function
def sample_noise(n_samples, latent_size):
    return torch.randn(n_samples, latent_size).to(device)

In [15]:


# Training loop
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        # Flatten the images
        images = images.reshape(batch_size, -1).to(device)
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train the discriminator
        discriminator.zero_grad()
        outputs_real = discriminator(images)
        d_loss_real = criterion(outputs_real, real_labels)
        d_loss_real.backward()

        noise = sample_noise(batch_size, latent_size)
        fake_images = generator(noise)
        outputs_fake = discriminator(fake_images.detach())
        d_loss_fake = criterion(outputs_fake, fake_labels)
        d_loss_fake.backward()
        d_loss = d_loss_real + d_loss_fake
        optimizer_d.step()

        # Train the generator
        generator.zero_grad()
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_g.step()

        if (i+1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], '
                  f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')

    # Save generated images
    with torch.no_grad():
        noise = sample_noise(25, latent_size)
        fake_images = generator(noise)
        save_image(fake_images.view(fake_images.size(0), 1, 28, 28),
                   os.path.join(sample_dir, f'fake_images-{epoch+1:03d}.png'))


Epoch [1/50], Step [200/938], D_loss: 0.1293, G_loss: 2.2481
Epoch [1/50], Step [400/938], D_loss: 0.1633, G_loss: 1.9351
Epoch [1/50], Step [600/938], D_loss: 0.0804, G_loss: 2.5711
Epoch [1/50], Step [800/938], D_loss: 0.0387, G_loss: 3.2769


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x392 and 784x256)

In [ ]:

# Evaluation metrics
def calculate_fid(real_features, generated_features):
    # calculate mean and covariance statistics
    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = generated_features.mean(axis=0), np.cov(generated_features, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2) ** 2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

def calculate_is(real_outputs, fake_outputs):
    p_real = np.mean(np.exp(real_outputs))
    p_fake = np.mean(np.exp(fake_outputs))
    is_score = p_real / p_fake
    return is_score

def calculate_mse(real_images, generated_images):
    return mean_squared_error(real_images, generated_images)

# Evaluation
generator.eval()
with torch.no_grad():
    # Generate a batch of fake images
    noise = sample_noise(len(mnist_data), latent_size)
    fake_images = generator(noise).cpu().numpy()

    # Flatten the real images
    real_images = mnist_data.data.numpy().reshape(len(mnist_data), -1)

    # Calculate FID
    fid_score = calculate_fid(real_images, fake_images)

    # Calculate IS
    is_score = calculate_is(real_images, fake_images)

    # Calculate MSE
    mse_score = calculate_mse(real_images, fake_images)

    print(f'FID: {fid_score:.4f}, IS: {is_score:.4f}, MSE: {mse_score:.4f}')
